In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pulp import *
from copy import deepcopy
while "notebooks" in os.getcwd():
    os.chdir("..")

from src.preprocessing.parser import Parser
from src.preprocessing.preprocessor import Preprocessor
from src.solvers.solution import Solution
from src.solvers.pulp_solver import PuLPSolver

from tqdm import tqdm
from typing import Dict

/tmp/ipykernel_115203/804485343.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
def generate(
    k : int,
    M : int,
    N : int,
    p_max : float,
    r_max : float,
):  
    data : Dict[int, pd.DataFrame] = {}
    for n in range(N):
        index = 0
        data[n] = pd.DataFrame([])
        for m in range(M):
            p_kmn = np.random.randint(1, p_max+1)
            r_kmn = np.random.randint(1, r_max+1)

            pair = {
                "k": [k],
                "m": [m],
                "n": [n],
                "p_k,m,n": [p_kmn],
                "r_k,m,n": [r_kmn]
            }
            
            index += 1
            data[n] = pd.concat([data[n], pd.DataFrame(pair, index= [index])])


    return data


In [6]:
d = generate(5,3,7,100,100)
d[0]

,k,m,n,"p_k,m,n","r_k,m,n"
1,5,0,0,49,92
2,5,1,0,51,34
3,5,2,0,38,1


In [7]:
# K = 10
# M = 2
# N = 4
# p_max = 50
# r_max = 100
# p = 100

# allocated_channels = set()
# data_rate = 0
# total_power = 0

# data : Dict[int , pd.DataFrame] = {n : pd.DataFrame([]) for n in range(N)}


# for k in range(K):
#     data_k = generate(k, M, N, p_max, r_max)

#     for n in range(N):
#         data[n] = pd.concat([data[n], data_k[n]]) 

#     pairs = []
#     for n in range(N):
#         if n in allocated_channels:
#             continue

#         pairs.append(data_k[n])

#     if len(pairs) == 0:
#         break

#     pairs = pd.concat(pairs)
#     pairs['e_k,m,n'] = pairs['r_k,m,n']/pairs['p_k,m,n']
#     sorted_pairs = pairs.sort_values(by='e_k,m,n', ascending=False)

#     power_used = 0

#     for idx, row in sorted_pairs.iterrows():
#         n = row['n']
#         p_kmn = row['p_k,m,n']
#         r_kmn = row['r_k,m,n']

#         if p_kmn + power_used <= p/K:
#             allocated_channels.add(n)
#             power_used += p_kmn
#             total_power += p_kmn
#             data_rate += r_kmn

#             if power_used == p/K:
#                 break

# data_rate, total_power

In [9]:
# compute e_average

def get_e_average(p_max, r_max) -> int:
    e = 0 
    for p_i in range(1,p_max +1):
        for r_i in range(1, r_max + 1):
            
            e+= r_i/p_i

    return e/(p_max*r_max)

In [55]:
def online_solution(
    K : int,
    M : int,
    N : int,
    p_max : int,
    r_max : int,
    p : float,
    tolerance : float = 0.6
):

    allocated_channels = set()
    data_rate = 0
    total_power = 0

    data : Dict[int , pd.DataFrame] = {n : pd.DataFrame([]) for n in range(N)}
    e_avg = get_e_average(p_max, r_max)

    for k in range(K):
        data_k = generate(k, M, N, p_max, r_max)

        for n in range(N):
            data[n] = pd.concat([data[n], data_k[n]]) 

            if n in allocated_channels:
                continue
            
            best_p, best_r = None, None
            for idx, row in data_k[n].iterrows():
                p_i = row['p_k,m,n']
                r_i = row['r_k,m,n']

                if p_i + total_power > p:
                    continue

                if best_p is None:
                    best_p = p_i
                    best_r = r_i

                    continue
                
                if compare(best_p, best_r, p_i, r_i, e_avg):
                    best_p = p_i
                    best_r = r_i

            if best_p is None:
                continue
            
            # We don't have choice if we are on the last user
            if k == K-1:

                allocated_channels.add(n)
                data_rate += best_r
                total_power += best_p

            else:

                if good_enough(best_p, best_r, p_max, r_max, e_avg, tolerance):
                    allocated_channels.add(n)
                    data_rate += best_r
                    total_power += best_p


    return data_rate, total_power, data, allocated_channels

def compare(
    p1 : int, 
    r1 : int, 
    p2 : int, 
    r2 : int, 
    e_average : float
):
    """Returns true if (p2, r2) is better than (p1, r1)

    Args:
        p1 (int): _description_
        r1 (int): _description_
        p2 (int): _description_
        r2 (int): _description_
    
    Returns:
        bool: If (p2, r2) is better than (p1,r1)
    """    
    return r2/p2 > r1/p1
    
    # return r2 - e_average*p2 >= r1 - e_average*p1

def good_enough(p, r, expected_e):
    """_summary_

    Args:
        p (_type_): _description_
        r (_type_): _description_
        expected_e (_type_): _description_

    Returns:
        bool: _description_
    """    
    return r/p >= expected_e
    


In [59]:
K = 10
M = 2
N = 4
p_max = 50
r_max = 100
p = 100

n_experiments = 100

channels = []
optimal_solution = []
sub_optimal_online_solution = []
for attempt in tqdm(range(n_experiments)):
    data_rate, total_power, data, allocated_channels = online_solution(
        K,
        M,
        N,
        p_max,
        r_max,
        p,
    )

    channels.append(allocated_channels)

    sub_optimal_online_solution.append(data_rate)

    lp_solver = PuLPSolver(
        K,
        M,
        N,
        p,
        data
    )

    lp_solver.solve()

    optimal_solution.append(lp_solver.get_data_rate())

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:09<00:00, 10.59it/s]


In [61]:
np.mean(sub_optimal_online_solution)/ np.mean(optimal_solution)

0.7020208014857612

In [53]:
lp_solver = PuLPSolver(
    K,
    M,
    N,
    p,
    data
)

lp_solver.solve()

lp_solver.get_data_rate()

383.0

In [54]:
data_rate, total_power

(331, 30)